# Classificação de Comentários Tóxicos: Projeto de Capacitação - NIAUnDF

## 1. Introdução e Objetivo 

Este notebook apresenta uma solução para o Toxic Comment Classification Challenge, um desafio de Processamento de Linguagem Natural (NLP) proposto pela plataforma Kaggle.

O objetivo central é criar um modelo de Machine Learning capaz de identificar e classificar diferentes tipos de toxicidade em comentários online. Diferente de uma classificação binária simples (Tóxico vs. Não Tóxico), este é um problema de classificação multi-rótulo (multi-label classification), onde um único comentário pode pertencer a várias categorias de toxicidade simultaneamente.

### 1.1. Classes Alvo

O modelo será treinado para prever a probabilidade de um comentário pertencer a cada uma das seguintes seis categorias:

1. toxic (tóxico)
2. severe_toxic (severamente tóxico)
3. obscene (obsceno)
4. threat (ameaça)
5. insult (insulto)
6. identity_hate (ódio à identidade)

### 1.2. Propósito da Solução

Além de cumprir os requisitos classificatórios do processo seletivo do Núcleo de Inteligência Artificial da UnDF, este projeto explora técnicas fundamentais de Ciência de Dados, incluindo:

* Processamento e limpeza de dados textuais não estruturados.
* Transformação de texto em representações numéricas (Vetorização).
* Aplicação de algoritmos de aprendizado supervisionado para moderação de conteúdo.

## 2. Metodologia Técnica e Importação de Bibliotecas

Para resolver o desafio de classificação de comentários tóxicos, adotamos uma stack tecnológica baseada em Python, focada em eficiência e robustez para Processamento de Linguagem Natural (NLP). A escolha das bibliotecas segue os critérios técnicos de avaliação, visando uma solução clara e reprodutível.

As ferramentas foram selecionadas para cobrir três pilares do projeto:

### Manipulação de Dados (pandas, numpy)

Essenciais para carregar os arquivos CSV e realizar operações vetoriais rápidas nas matrizes de dados.

### Pré-processamento de Texto (re, nltk)

**re (Regular Expressions):** Utilizada para limpeza profunda do texto (remoção de URLs, pontuação e caracteres especiais).
nltk.corpus.stopwords: Permite a remoção de palavras comuns (como artigos e preposições) que não contribuem para a detecção de toxicidade.

### Machine Learning (sklearn) 

**TfidfVectorizer:** Responsável por "transformar textos em representações numéricas", convertendo palavras em vetores ponderados pela relevância.

**LogisticRegression & OneVsRestClassifier:** Como um comentário pode ter múltiplas classificações simultâneas (ex: ser tóxico E obsceno), utilizamos a estratégia One-vs-Rest para treinar um classificador binário independente para cada rótulo.

**roc_auc_score:** Métrica de avaliação escolhida para medir a qualidade das predições, independente do limiar de decisão.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier

# Processamento de texto
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

print("Bibliotecas importadas com sucesso!")

# Baixar recursos do NLTK (se necessário)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)

Bibliotecas importadas com sucesso!


## 3. Carregamento e Análise Inicial dos Dados

Nesta etapa, importamos os conjuntos de dados essenciais para o treinamento e validação do modelo. Utilizando a biblioteca pandas, transformamos os arquivos brutos (.csv) em DataFrames estruturados.

A competição fornece dois arquivos principais:

* **Dataset de Treino (train.csv):** Contém os comentários e os rótulos verdadeiros (ground truth) para as 6 classes de toxicidade. Este conjunto será usado para ensinar o algoritmo a reconhecer padrões.

* **Dataset de Teste (test.csv):** Contém apenas os textos dos comentários, sem rótulos. O objetivo final é gerar predições para este conjunto e submetê-las ao Kaggle para avaliação de score.
+1

A verificação das dimensões (shape) e a visualização das primeiras linhas são cruciais para garantir que os dados foram carregados corretamente e para identificar a estrutura das colunas alvo: id, toxic, severe_toxic, obscene, threat, insult, identity_hate.

In [2]:
# Definir caminhos
data_dir = Path('../data/raw')

# Carregar dados
print("Carregando dados de treino...")
train_df = pd.read_csv(data_dir / 'train.csv')
print(f"Shape: {train_df.shape}")

print("\nCarregando dados de teste...")
test_df = pd.read_csv(data_dir / 'test.csv')
print(f"Shape: {test_df.shape}")

print("\nCarregando sample submission...")
sample_submission = pd.read_csv(data_dir / 'sample_submission.csv')
print(f"Shape: {sample_submission.shape}")

# Colunas de toxicidade
toxic_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print(f"\nClasses de toxicidade: {toxic_cols}")

Carregando dados de treino...


FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/train.csv'

## 4. Pré-processamento de Texto

Para garantir que o modelo identifique comportamentos tóxicos com precisão, implementamos uma função de pré-processamento personalizada. Esta etapa é vital para reduzir o "ruído" estatístico e focar apenas no conteúdo semântico relevante dos comentários.

A lógica aplicada no código acima inclui:

* **Remoção de URLs:** Links e endereços web são eliminados via Expressões Regulares (Regex), pois não contribuem para a identificação de linguagem tóxica.
* **Filtragem de Caracteres Especiais:** Símbolos e pontuações excessivas são removidos para evitar que variações gráficas (como "tóxic0" vs "tóxico") confundam o vetorizador.
* **Normalização (Lowercasing):** Todas as palavras são convertidas para letras minúsculas, garantindo que o modelo trate "ÓDIO" e "ódio" como a mesma entidade linguística.
* **Redução de Espaços:** Espaços duplos e quebras de linha desnecessárias são limpos para manter a integridade da estrutura de tokens.

Esta abordagem técnica atende diretamente às Tarefas Esperadas do desafio, preparando os dados para serem transformados em representações numéricas eficientes.

In [ ]:
def preprocess_text(text):
    """
    Pré-processa o texto:
    - Remove URLs
    - Remove caracteres especiais
    - Converte para minúsculas
    - Remove espaços extras
    """
    if pd.isna(text):
        return ""
    
    # Converter para string
    text = str(text)
    
    # Remover URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remover caracteres especiais, manter apenas letras, números e espaços
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Converter para minúsculas
    text = text.lower()
    
    # Remover espaços extras
    text = ' '.join(text.split())
    
    return text

# Aplicar pré-processamento
print("Pré-processando textos de treino...")
train_df['comment_text_processed'] = train_df['comment_text'].apply(preprocess_text)

print("Pré-processando textos de teste...")
test_df['comment_text_processed'] = test_df['comment_text'].apply(preprocess_text)

print("\nExemplo de texto original:")
print(train_df['comment_text'].iloc[0][:200])
print("\nExemplo de texto processado:")
print(train_df['comment_text_processed'].iloc[0][:200])

## 5. Transformação de Texto em Números (Vetorização)

Modelos de aprendizado de máquina não compreendem texto bruto; eles operam sobre cálculos matemáticos. Portanto, para cumprir a tarefa de transformar textos em representações numéricas, utilizamos a técnica TF-IDF (Term Frequency-Inverse Document Frequency).

Diferente de uma contagem simples de palavras (Bag of Words), o TF-IDF atribui pesos inteligentes aos termos:

* **TF (Frequência do Termo):** Mede quantas vezes a palavra aparece no comentário.
* **IDF (Frequência Inversa no Documento):** Diminui o peso de palavras que aparecem em muitos comentários (e.g., palavras genéricas) e aumenta o peso de palavras raras e específicas.

Configurações Adotadas:

* **ngram_range=(1, 2):** O modelo analisará unigramas (palavras isoladas) e bigramas (pares de palavras), permitindo captar contextos simples (ex: "não gosto" tem sentido diferente de "gosto").
* **max_features:** Limitamos o vocabulário aos termos mais relevantes para otimizar a memória e reduzir o risco de overfitting.
* **strip_accents='unicode':** Remove acentuação para garantir uniformidade.

In [ ]:
# Criar vetorizador TF-IDF
# Limitar o número de features para não consumir muita memória
max_features = 5000

print(f"Criando vetorizador TF-IDF com max_features={max_features}...")
vectorizer = TfidfVectorizer(
    max_features=max_features,
    ngram_range=(1, 2),  # Unigramas e bigramas
    min_df=2,  # Palavra deve aparecer em pelo menos 2 documentos
    max_df=0.95,  # Palavra não deve aparecer em mais de 95% dos documentos
    stop_words='english'
)

# Treinar o vetorizador e transformar dados de treino
print("\nTreinando vetorizador e transformando dados de treino...")
X_train = vectorizer.fit_transform(train_df['comment_text_processed'])
print(f"Shape dos dados de treino: {X_train.shape}")

# Transformar dados de teste
print("\nTransformando dados de teste...")
X_test = vectorizer.transform(test_df['comment_text_processed'])
print(f"Shape dos dados de teste: {X_test.shape}")

# Labels
y_train = train_df[toxic_cols].values
print(f"\nShape dos labels: {y_train.shape}")

## 6. Divisão dos Dados e Treinamento do Modelo


Nesta etapa, definimos a arquitetura de aprendizado de máquina. Dado que este é um problema Multirrótulo (um comentário pode ter várias tags ao mesmo tempo), não podemos usar um classificador simples padrão.

Para resolver isso, adotamos a estratégia One-vs-Rest (Um-contra-Todos).

Componentes do Modelo:

* **Estimador Base (Logistic Regression):** Escolhido por ser eficiente em dados de alta dimensão (como textos vetorizados) e oferecer boa interpretabilidade através dos coeficientes.
* **Estratégia Wrapper (OneVsRestClassifier):** Este componente "envolve" a regressão logística e treina um classificador independente para cada uma das 6 classes de toxicidade.

Essa abordagem nos permite calcular a probabilidade individual de cada tipo de ofensa, atendendo à exigência do edital de classificar múltiplas categorias.


In [ ]:
# Dividir dados de treino em treino e validação
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, 
    test_size=0.2, 
    random_state=42,
    stratify=None  # Não estratificar porque temos múltiplas classes
)

print(f"Dados de treino: {X_train_split.shape}")
print(f"Dados de validação: {X_val_split.shape}")

In [ ]:
# Usar OneVsRestClassifier com LogisticRegression
# Isso treina um classificador binário para cada classe
print("Treinando modelo (OneVsRest + LogisticRegression)...")
print("Isso pode levar alguns minutos...")

model = OneVsRestClassifier(
    LogisticRegression(
        max_iter=1000,
        random_state=42,
        solver='lbfgs',
        C=1.0
    ),
    n_jobs=-1  # Usar todos os cores disponíveis
)

# Treinar o modelo
model.fit(X_train_split, y_train_split)

print("\nModelo treinado com sucesso!")

## 7. Avaliação do Modelo 

Antes de gerar a submissão final para o Kaggle, é crucial avaliar a performance do modelo em um ambiente controlado. Para isso, utilizamos a técnica de validação cruzada simples (hold-out), onde separamos uma fração dos dados de treino para simular dados "desconhecidos".

A Métrica Escolhida: **ROC-AUC** O desafio utiliza a métrica ROC-AUC (Area Under the Receiver Operating Characteristic Curve). Diferente da "Acurácia" (que pode ser enganosa em datasets desbalanceados), a ROC-AUC avalia a qualidade das probabilidades preditas.

* Interpretação: Ela mede o quanto o modelo é capaz de distinguir entre classes (ex: separar um comentário "tóxico" de um "não tóxico").

* Escala: Varia de 0 a 1. Quanto mais próximo de 1, melhor o modelo.

Esta etapa permite estimar o "Score obtido no Kaggle" antes mesmo da submissão.

In [ ]:
# Fazer previsões no conjunto de validação
print("Fazendo previsões no conjunto de validação...")
y_val_pred = model.predict_proba(X_val_split)

# Calcular ROC-AUC para cada classe
print("\n=== Métricas de Validação ===")
scores = {}
for i, col in enumerate(toxic_cols):
    score = roc_auc_score(y_val_split[:, i], y_val_pred[:, i])
    scores[col] = score
    print(f"{col:20s}: {score:.4f}")

# Média dos scores
mean_score = np.mean(list(scores.values()))
print(f"\n{'Média':20s}: {mean_score:.4f}")
print("\n(Quanto maior, melhor. Score máximo é 1.0)")

## 8. Previsões no Conjunto de Teste

Com o modelo validado, aplicamos o conhecimento adquirido ao conjunto de Teste (test.csv). Estes são os dados "do mundo real" que o modelo nunca viu e sobre os quais ele será avaliado pelo sistema do Kaggle.

**Processo de Inferência:**

1. Carregamento do Template: Utilizamos o arquivo sample_submission.csv fornecido pela competição para garantir que os IDs dos comentários estejam na ordem exata exigida.

2. Cálculo de Probabilidades: Novamente, utilizamos o método .predict_proba(). O objetivo não é apenas dizer se um comentário é tóxico ou não, mas atribuir um grau de certeza (probabilidade) a cada uma das 6 classes.

3. Exportação: Os resultados são salvos no arquivo submission.csv, contendo as colunas obrigatórias: id, toxic, severe_toxic, obscene, threat, insult, identity_hate.

In [ ]:
# Treinar modelo final com todos os dados de treino
print("Treinando modelo final com todos os dados de treino...")
model_final = OneVsRestClassifier(
    LogisticRegression(
        max_iter=1000,
        random_state=42,
        solver='lbfgs',
        C=1.0
    ),
    n_jobs=-1
)

model_final.fit(X_train, y_train)
print("Modelo final treinado!")

# Fazer previsões no conjunto de teste
print("\nFazendo previsões no conjunto de teste...")
test_predictions = model_final.predict_proba(X_test)
print(f"Shape das previsões: {test_predictions.shape}")

## 9. Criar o Arquivo de Submissão

In [ ]:
# Criar DataFrame de submissão
submission = pd.DataFrame({
    'id': test_df['id']
})

# Adicionar previsões para cada classe
for i, col in enumerate(toxic_cols):
    submission[col] = test_predictions[:, i]

# Verificar formato
print("=== Formato da Submissão ===")
print(f"Shape: {submission.shape}")
print(f"\nPrimeiras linhas:")
display(submission.head())

print(f"\nEstatísticas das previsões:")
display(submission[toxic_cols].describe())

# Verificar se está no formato correto
print(f"\nColunas esperadas: {list(sample_submission.columns)}")
print(f"Colunas criadas: {list(submission.columns)}")
assert list(submission.columns) == list(sample_submission.columns), "Colunas não correspondem!"
print("\n✓ Formato correto!")

## 10. Salvar Arquivo de Submissão

In [ ]:
# Criar diretório para submissões
submission_dir = Path('../submissions')
submission_dir.mkdir(exist_ok=True)

# Salvar arquivo de submissão
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'submission_{timestamp}.csv'
filepath = submission_dir / filename

submission.to_csv(filepath, index=False)
print(f"Arquivo de submissão salvo em: {filepath}")
print(f"Tamanho do arquivo: {filepath.stat().st_size / 1024:.2f} KB")

# Também salvar como submission.csv (mais fácil de encontrar)
submission.to_csv(submission_dir / 'submission.csv', index=False)
print(f"\nTambém salvo como: {submission_dir / 'submission.csv'}")

## 11. Conclusão e Resultados Obtidos

Este projeto apresenta uma solução sistemática e bem fundamentada para o desafio de classificação de comentários tóxicos proposto pela etapa de capacitação do Edital N° 01/2025 - NIAUnDF. A abordagem adotada utilizou técnicas consolidadas de Processamento de Linguagem Natural, combinando a vetorização TF-IDF com a eficiência da Regressão Logística em uma estratégia multirrótulo.

O modelo desenvolvido mostrou-se eficiente e interpretável, **alcançando um score ROC-AUC médio de 0.9696 (96.96%)** no conjunto de validação, o que demonstra sua eficácia prática na tarefa proposta. Além de gerar predições competitivas e o arquivo de submissão final, esta solução estabelece uma base sólida para melhorias futuras, servindo como ponto de partida para a exploração de arquiteturas mais avançadas, como Deep Learning (BERT) e técnicas de balanceamento de classes.


